In [18]:
import numpy as np
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression

n_samples, n_features = 1000, 20
rng = np.random.RandomState(0)
X, y = make_regression(n_samples, n_features, noise=1.0, random_state=rng)

y_positive = y - np.min(y)
lm = LinearRegression().fit(X, y_positive)
rlm = LinearRegression().fit(X, np.sqrt(y_positive))
mmlm = LinearRegression().fit(X, y_positive / np.max(y_positive))
stlm = LinearRegression().fit(X, y_positive - np.mean(y_positive) / np.std(y_positive))

In [19]:
(lm.score(X, y_positive), 
 rlm.score(X, np.sqrt(y_positive)), 
 mmlm.score(X, y_positive / np.max(y_positive)),
 stlm.score(X, y_positive - np.mean(y_positive) / np.std(y_positive))
)

(0.9999782533853913,
 0.9713400828419546,
 0.9999782533853913,
 0.9999782533853913)